#0 - General concepts (for instance, what is artificial intelligence, machine learning, deep learning)



### Artificial Intelligence

The attempt to build artificially intelligent systems have been around since the dawn of man. One could argue that just about any machine possible is actually an example of artifical intelligence systems, as they attempt to automatically complete tasks that humans are meant to do. One could, however, come to a less broad definition of AI by relegating it to systems that solve "decision problems" for humans.

Oddly enough, this actually makes it just as difficult to nail down the true definition of artificial intelligence. For example, to complete merge sort, one must "decide" as to if one number is greater than another. Therefore, by this definition, simple freshman year algorithms could be considered AI. Therefore, we should try to narrow our definition even further. 

What problems do we want solved by AI? When we examine problems like stock prediction, car driving, or image classification, one might note these problems ought be classified in the "non-deterministic" class of problems. These problems are likely not solvable 100% of time with deterministic algorithm, and therefore require some kind of hueristic to solve optimally with the highest degree of accuracy. 

This, however, leaves out problems that CAN be solved in polynomial time deterministically, such as maze solving. Algorithms such as A* will always find the shortest path in some polynomial (non-exponential) time, and therefore our previous definition would leave them out. At these point, out attempt to dedfine Artificial Intelligence becomes circular. There is no objective reason why maze solving is AI, but list sorting is not. 

Therefore, it might be more useful not to try and define AI by the type of strategy the system replicates, rather than the atual problems used. List sorting, even by humans, does not use any "human like" characterisitcs, and instead just mimicks computers. Most humans default to insertion sort, but still follow a "computer-like" algorithm. However, when humans are presented with a maze, they intead fall back to using their memory, understanding of the enviroment, and other hueristics. Humans classify dogs and cats by use of visual perception, not an objecitve analysis of the actual input to their eyes. AI, therefore, is an attempt to find objective computable methods to replicate the types of hueristic measures that result from human perception.

### Machine Learning

Machine learning, as a subset of AI, is a modification of how the systems are replicated. In a way, one could consider machine learning to be "second order" artificial intelligence. Instead of programming the method for replicating the human action or perception, we instead opt program the machine to design that system itself (by presumably mimiking the ways in which humans themselves learn). The analogy is that instead of giving the computer the rules, and having it process the inputs and the outputs, the computer is given the inputs and the outputs and is programmed to learn the rules. 

The issue with this, however, is similar to the issues with most current arificial intelligence. A computer using linear regression to determine a housing price, or using logistic regression to classify dogs and cats, do not at all replicate the ways in which humans learn to complete these tasks. Humans do not typically need 60,000 examples in order to learn what something is. In fact, humans often only need one or two examples. Some work in the field of computational cognitive science has attempted fix this issue.



https://github.com/ucfai/intelligence/blob/master/fa19/2019-11-12-machines-learn-think-people/machines-learn-think-people.pdf

### Deep Learning

Deep learning is a strategy for machine learning. While there isn't a strict definition of "deep learning", as it actually encompasses many different types of systems, the simple way to define it is learning using differentiable computation graphs. Each "node" on the graph represents some sort of differentible transformation on the data

These transformations can be nearly anything, but the most common is a simple matrix multiplication with a matrix of "weights". This transformation is called a "dense" layer, and is meant to replicate nuerons in a brain. However, researchers have found over time that this model can be improved by simply looking for more robust transformations, and not attempting to replicate human biology for its own sake. Exmaples of this include the convolutional layer, the recurrant layer, and attention transformers, the later of which in no way mirrors brain biology but produces state of the art results in language and time series tasks.

https://arxiv.org/abs/1706.03762

# Building a model


Building a deep learning model in keras is quite simplistic. Let's load in a dataset

In [0]:
import tensorflow as tf
from tensorflow import keras

import matplotlib.pyplot as plt
import numpy as np

import matplotlib.pyplot as plt
from keras.datasets import cifar10

from keras.layers import Conv2D, MaxPooling2D, Dense, Activation, Flatten, Dropout
from keras.models import Sequential
from keras.optimizers import SGD

(train_images, train_labels), (test_images, test_labels) = cifar10.load_data()
num_classes = 10
class_names = ['airplan', 'automobile', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']

In [0]:
train_labels = keras.utils.to_categorical(train_labels, num_classes)
test_labels = keras.utils.to_categorical(test_labels, num_classes)


train_images = train_images.astype('float32')
test_images = test_images.astype('float32')
train_images /= 255
test_images /= 255

The following model is a very basic example of a convolutional network.

### Sequential()
This is simply a keras object that describes a computational graph with one transformation after another

### Conv2D
The most basic of convolutions. A filter is passed over the image to map one grid of real numbers to another, typically to reduce the size and to extract important features

### MaxPooling2D
Devides the input into squares, and only takes the highest value input in each of the squares as the result

### Activation 
Maps the input to some real value between {0, 1} (or -1 and 1). It can be any range but typically its between 0 and 1.

### Flatten
Takes a grid of real numbers and maps it to a simple vector.

In [0]:
model = Sequential()
model.add(Conv2D(32, (3,3), padding='same', input_shape=(32, 32, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Conv2D(32, (3,3), padding='same', ))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Activation('relu'))
model.add(Flatten())
model.add(Dense(128))
model.add(Activation('relu'))
model.add(Dense(32))
model.add(Activation('relu'))
model.add(Dense(num_classes, activation='softmax'))


# 2. Comping a model (for instance, you can talk here about optimizers, learning rate etc.)

When we compile our model, there are a few parameters we need to spesify

### Optimizer
The actual algorithm for performing backpropagation on our network. There are different forms of this including

#### Stochastic Gradient Descent 
An an optmizer that simply relies on a batch of training data in order to produce the gradient to subtract from the weights

#### Adam
A second order optimization method that adjusts the learning rate dynamically during training in order to optimize learning

#### AdaBoost
A less popular optimizer typically used on descision trees, but still usable on deep learning models

### Learning Rate
The constant that multiplies the gradient. This is applied to reduce the speed change of the weights to prevent the model from overshooting a optimal minimum.

### Loss
The actual function for comparing the output of the network to the ground truth of the training data. Like all aspects of our computational graph, this must also be differentiable. 

In [0]:
# compile model
opt = SGD(lr=0.001, momentum=0.9)
model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])

# 3. Training a model (for instance, you can talk about overfitting/underfitting)

As mentioned before, deep learning models are trained on extremly large amounts of data in order to apply the prefered optimizer on the millions of parameters. Some of the hyper parameters to consider before training include the epochs, which represents the number of times that the entirty of the data will be pushed through the model.

When training, there are typically two situations that you want to avoid

### Underfitting

Underfitting is simply when your model does not learn. There are significantly more ways that this can occur than the inverse, mostly because machine learning is actually pretty hard. The most common way that a model can fail to learn is simply because the model is not complex enough to understand the function in question. You are not going to be able to solve Go with Logistic Regression. No matter how much data you feed to logistic regresion, it is not going to be complex enough to play on the most complex games on the earth.

Additionally, underfitting often occurs in absense of sufficient data. In the end, there really isn't anythign that intelligent about deep learning, All it does  it try to adjust linear equations using calculus to try and mimick the output of the data you give it. Therefore, the more sparse your dataset, the more likely that the function that your model learns is not going to actually generalize to the dataset in general. 

### Overfitting

Despite the fact that underfitting is much more common, overfitting is often the problem that gets a lot more attention. In essense, overfitting results due to the same issue with deep learning that we have mentioned many times already: it is simply a very dumb attempt at mimickry. What the deep learning model is actually doing is just trying to replicate the output of the training data as much as possible. Therefore, models that are too complex may have so many parameters that they are capable of simply creating a perfect model of the training data. The problem with this approach is that we are not trying to create a model of the training data, we are training to create a model of the function that PRODUCED the training data. Other than simply making the model less complex, this problem can be fixed by introducing "Dropout layers", which simple introduce some random noise into the system by systematically "forgetting" some data during training. This way, the model is encouraged to understand the underlying function rather than just memorizing the training data.

In [0]:
# train the model
epochs = 50
history = model.fit(train_images, 
                      train_labels, 
                      epochs=epochs,  
                      validation_data=(test_images, test_labels),
                      batch_size = 32)


#4.  Finetuning  a pretrained model (describe how you proceed)



LIke most things in computer science, it is not worth reinventing the wheel. If some sort of model has already been built for your problem, it may be worth using that already trained model. This isn't to say that there is no point is reexamining already sovled problems, but rather that it is reasonable to believe that the problem you are attempting to solve may be made up of problems previously solved.

Take classifying cats and dogs for example. One would believe that learning edges, circles, lines, etc is required to begin recognizing a cat. Additionally, knowing the difference between tail lengths, whiskers, etc, may also be part of this problem. Therefore, one might find that using weights from a model trained on animals in general may lend well to learning the difference between cats and dogs.

However, that pre-trained animal model contains may layers that look for features that are totally irrelevent to the particular problem of cats and dogs. Perhaps there is a layer that learned to look for the presense of wings, or another that looked to see how many eyes the creature had. These features are useless to somebody trying to clasify furry friends, and therefore it would be useful to try "freezing" layers of the pre-trained model to see if this would speed up performance. 

Additionally, for problems in the realm of reinforcement learning, there is often not a reaosn to even bother changing a model, as they are meant to generalize to as many problems as possible. For example, in my research, we found that using Promixal Policy Optimization (PPO), with a model meant for playing the game DOTA2 worked just as well for the problem of epidemic control.

https://www.semanticscholar.org/paper/Reinforcement-Learning-for-Optimal-Control-of-Kerrigan/2bcb8babe485a55a60b809e41839572ed219586c

Enjoy reading :)